In [1]:
import json

In [2]:
import os

In [3]:
import pandas as pd

In [4]:
result_directories = [
#     "baseline_results/baseline_results_10_dims",
#     "baseline_results/baseline_results_50_dims",
#     "baseline_results/baseline_results_100_dims",
#     "baseline_results/baseline_results_gurobi",
#     "baseline_results/baseline_results_200_dims",
    "baseline_results/baseline_results_Levy_7200_to",
]

In [5]:
def extract_function(file_name):
    candidates = ["Michalewicz", "Levy", "Ackley"]
    for cand in candidates:
        if cand in file_name:
            return cand
    
    assert False, "Function is not recognized from candidates {} for file {}.".format(candidates, file_name)

In [6]:
def extract_dimension(file_name):
    dimension = file_name.split("_")[1]
    assert str.isdigit(dimension), "Dimension {} is not an integer.".format(dimension)
    return int(dimension)

In [7]:
def extract_solver(file_name):
    candidates = ["scipy", "gurobi"]
    for cand in candidates:
        if cand in file_name:
            return cand
    
    assert False, "Solver is not recognized from candidates {} for file {}.".format(candidates, file_name)

In [8]:
def extract_algorithm(file_name):
    candidates = ["direct", "shgo", "differential_evolution", "dual_annealing", "brute", "basinhopping"]
    for cand in candidates:
        if cand in file_name:
            return cand
    
    assert False, "Algorithm is not recognized from candidates {} for file {}.".format(candidates, file_name)

In [9]:
def extract_seed(file_name):
    seed = file_name.strip(".json").split("_")[-1]
    assert str.isdigit(seed), "Seed {} is not an integer.".format(seed)
    return int(seed)

In [10]:
results = []

for result_dir in result_directories:
    path = os.path.join(result_dir, "results")
    for fn in os.listdir(path):
        entry_dict = {}
        function = extract_function(fn)
        solver = extract_solver(fn)
        if solver == "gurobi":
            algorithm = None
        else:
            algorithm = extract_algorithm(fn)
        dimension = extract_dimension(fn)
        seed = extract_seed(fn)
        
        info_dict = {
            "function": function,
            "solver": solver,
            "algorithm": algorithm,
            "dimension": dimension,
            "seed": seed,
        }
        with open(os.path.join(path, fn)) as f:
            result_dict = json.load(f)
        entry_dict.update(info_dict)
        entry_dict.update(result_dict)
        
        results.append(entry_dict)

In [11]:
for result in results:
    if result['algorithm'] == None:
        result['algorithm'] = 'gurobi'

In [12]:
df = pd.DataFrame(results)
df['timeout_reached'] = df['timeout_reached'].fillna(False)

In [13]:
result_df = df.sort_values(by=['dimension', 'function', 'solver', 'algorithm', 'seed'])

In [14]:
result_df

,function,solver,algorithm,dimension,seed,x,obj_val,time_elapsed,timeout_reached
32,Levy,gurobi,gurobi,100,0,"[1.0000038162216711, 1.0, 1.0, 1.0, 1.0, 1.0, ...",-4.562944e-07,11.858348,False
59,Levy,gurobi,gurobi,100,1,"[1.0000038162216711, 1.0, 1.0, 1.0, 1.0, 1.0, ...",-4.562944e-07,12.292553,False
24,Levy,gurobi,gurobi,100,2,"[1.0000038162216711, 1.0, 1.0, 1.0, 1.0, 1.0, ...",-4.562944e-07,12.160086,False
36,Levy,gurobi,gurobi,100,3,"[1.0000038162216711, 1.0, 1.0, 1.0, 1.0, 1.0, ...",-4.562944e-07,11.792055,False
41,Levy,gurobi,gurobi,100,4,"[1.0000038162216711, 1.0, 1.0, 1.0, 1.0, 1.0, ...",-4.562944e-07,11.473904,False
50,Levy,scipy,basinhopping,100,0,"[1.4780111553010977, 2.6696683769737475, 1.801...",6.336665e+04,12.977481,False
9,Levy,scipy,basinhopping,100,1,"[-0.8429698611808254, 4.308468889492457, -10.8...",6.728884e+04,12.184939,False
51,Levy,scipy,basinhopping,100,2,"[-1.8444784594659875, -8.51480716132838, 0.868...",6.750075e+04,11.434944,False
15,Levy,scipy,basinhopping,100,3,"[-0.2007073941437147, 3.975170076003068, -6.84...",6.213343e+04,13.984549,False
20,Levy,scipy,basinhopping,100,4,"[9.589958596449899, 3.1091506556874724, 10.621...",7.007667e+04,16.769771,False


In [15]:
statistics_df = df.groupby(by=['dimension', 'function', 'solver', 'algorithm']).agg(
    obj_mean=('obj_val', 'mean'),
    obj_min=('obj_val', 'min'),
    obj_max=('obj_val', 'max'),
    time_mean=('time_elapsed', 'mean'),
    time_min=('time_elapsed', 'min'),
    time_max=('time_elapsed', 'max'),
    num_seeds=('seed', 'count'),
    num_timeout=('timeout_reached', 'sum'),
).reset_index()

In [16]:
statistics_df

,dimension,function,solver,algorithm,obj_mean,obj_min,obj_max,time_mean,time_min,time_max,num_seeds,num_timeout
0,100,Levy,gurobi,gurobi,-4.562944e-07,-4.562944e-07,-4.562944e-07,11.915389,11.473904,12.292553,5,0
1,100,Levy,scipy,basinhopping,6.607327e+04,6.213343e+04,7.007667e+04,13.470337,11.434944,16.769771,5,0
2,100,Levy,scipy,differential_evolution,6.618145e+04,6.436728e+04,6.764253e+04,1388.001272,1288.571079,1514.455667,5,0
3,100,Levy,scipy,direct,5.315829e+04,5.315829e+04,5.315829e+04,7.219747,5.956666,10.845012,5,0
4,100,Levy,scipy,dual_annealing,1.636139e+04,1.636138e+04,1.636139e+04,373.820010,234.408651,735.681609,5,0
5,100,Levy,scipy,shgo,NaN,NaN,NaN,7200.186159,7200.117647,7200.215185,5,5
6,200,Levy,gurobi,gurobi,-4.124202e-07,-4.124202e-07,-4.124202e-07,7.737716,7.437635,7.993145,5,0
7,200,Levy,scipy,basinhopping,6.994684e+05,6.619398e+05,7.340133e+05,26.831743,22.127060,31.288245,5,0
8,200,Levy,scipy,differential_evolution,6.562541e+05,6.493422e+05,6.658870e+05,3390.027804,3045.266311,4201.712123,5,0
9,200,Levy,scipy,direct,6.223000e+05,6.223000e+05,6.223000e+05,12.427938,10.749344,16.342058,5,0


In [25]:
result_df.to_csv("result_with_200.csv", index=False)
statistics_df.to_csv("stats_with_200.csv", index=False)

In [17]:
result_df.to_csv("Levy_results_7200_timeout.csv", index=False)
statistics_df.to_csv("Levy_stats_7200_timeout.csv", index=False)